# A short introduction to machine learning

Machine learning is really just fancy statistics. 
Still, machine learning is fun, and here we’ll take a look at what we can do.
There are three major types of machine learning:
- Supervised learning 
- Unsupervised learning 
- Reinforcement learning 

Here, we will only discuss supervised and unsupervised learning.

But first, we will need to import some libraries for machine learning.
- scikit-learn (`sklearn`) is a toolkit for machine learning
- matplotlib is a library for plotting and visualizing results
- numpy is a package for scientific computing with Python 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, make_classification, make_blobs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import numpy as np

# define colors
colors = plt.cm.RdBu
colors_bright = ListedColormap(['#FF0000', '#0000FF'])

## Supervised learning: classification

By supervision we mean human supervision.
Supervised learning works with labeled (annotated) data. The labels must be made manually by people, and this can require a lot of work for large data sets.

A common machine learning task is *classification* of items.
That is, determining which group or class an item belongs to.
*Neural networks* are powerful classifiers that can be used with complex data sets. But there are also many simpler classifiers.
We will try a couple of different non-neural classifiers, and then try neural networks in an exercise.

Before we start coding the machine learning part, we will need a way to visualize the results. 
Below, we define a function to plot the decision boundary produced by classifier.
The details of this function are not important, so you don't need to spend much time understanding it.

In [ ]:
def plot_boundary(classifier, data, datasets):
    X_train, X_test, y_train, y_test = datasets
    score = classifier.score(X_test, y_test)
    figure = plt.figure(figsize=(7, 7))
    h = .02  # step size in the mesh
    x_min, x_max = data[:, 0].min() - .5, data[:, 0].max() + .5
    y_min, y_max = data[:, 1].min() - .5, data[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(classifier, "decision_function"):
        Z = classifier.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = classifier.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=colors, alpha=.8)

    # Plot the training points
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=colors_bright,
               edgecolors='k')
    # Plot the testing points
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=colors_bright,
               edgecolors='k', alpha=0.6)
    
    #add labels
    plt.xlabel('lazy - active')
    plt.ylabel('size')
    plt.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score),
             size=15, horizontalalignment='right')
    plt.show()

Next, we need some data to work with.
As an example, we will use a simulated, random data set.

In [ ]:
data, labels = make_classification(n_features=2, n_redundant=0, n_informative=2,
                                   random_state=2, n_clusters_per_class=1)

We can examine the data:

In [ ]:
print(data)

In [ ]:
print(labels)

And we can visualize them with matplotlib.
The plot shows that this data set is nearly linearly separable.
The blue class has a couple of extreme values or *outliers*.

In [ ]:
# Plot the data
plt.scatter(data[:, 0], data[:, 1], c=labels, cmap=colors_bright)

Now, we define a function that tests given classifier with our data set.

When training a supervised machine learning algorithm, we want to be able to verify that the resulting model works on new data. We need to test that the model generalizes to unseen data.
Therefore, we always split the data set into training and test sets.

Then we train the classifier on the training set. This is called *fitting* the model.

In [ ]:
def run_classifier(classifier, X, y):
    datasets = train_test_split(X, y, test_size=.4, random_state=42)
    X_train, X_test, y_train, y_test = datasets
    classifier.fit(X_train, y_train)
    plot_boundary(classifier, X, datasets)

### Support-vector machine (SVM) classifier

Let's try a linear SVM classifier.
The accuracy on the test set is listed in the lower right corner of the plot.

In [ ]:
classifier = SVC(kernel="linear", C=0.025)
run_classifier(classifier, data, labels)

We can use this classifier on new data. This is also called *predicting* their classes or labels.

In [ ]:
print(classifier.predict([[1.1, 2.0],
                          [-1, -2]]))

### k-Nearest Neighbors (kNN)

k-Nearest Neighbors (kNN) is an efficient classifier.
kNN assigns a data point the same class as the majority of its neighbors.
The value k specifies how many neighbors should be considered.

In [ ]:
classifier = KNeighborsClassifier(3)
run_classifier(classifier, data, labels)

### Linearly separable data

The data we have worked with so far, are (nearly) linearly separable.
Let's try the SVM and kNN classifiers with a data set that is not linearly separable.

In [ ]:
data2, labels2 = make_circles(noise=0.2, factor=0.5)

The linear SVM classifier works poorly with this data set, and yields a low accuracy.

In [ ]:
classifier = SVC(kernel="linear", C=0.025)
run_classifier(classifier, data2, labels2)

 kNN, on the other hand, works well with data that is not linearly separable.

In [ ]:
classifier = KNeighborsClassifier(3)
run_classifier(classifier, data2, labels2)

## Unsupervised clustering

In unsupervised learning there are no labels.
The task is to find patterns in the data, for example by clustering them.
We avoid the human effort to label the data but get less information as a result.

First, let's plot the unlabeled data.

In [ ]:
figure = plt.figure(figsize=(7, 7))
plt.scatter(data[:, 0], data[:, 1])#, s=5, linewidth=0)

### k-means clustering

k-means clustering is a common clustering algorithm.
It takes the parameter *k*, which specifies the number of clusters we want to find.
In other words, the user needs to know beforehand, *a priori*, how many clusters they wish to look for.
We can of course try different values for k, to see which works better.


In [ ]:
Kmean = KMeans(n_clusters=2)
Kmean.fit(data)
clusters = Kmean.predict(data)
figure = plt.figure(figsize=(7, 7))
plt.scatter(data[:, 0], data[:, 1])#, s=5)#, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='x')

## Exercises

### Exercise: Decision tree classifier

Below, we define a decision tree classifier.
Use this classifier to classify both our data sets, data and data2.
Discuss and explain the results, especially the decision boundary.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_classifier = DecisionTreeClassifier(max_depth=5)
# your code here

### Exercise: Neural network classifier

Below, we define two neural network classifiers.
Use these classifiers to classify both our data sets, data and data2.
Compare and discuss the results. What does the decision boundary
tell us about these classifiers?

In [ ]:
single_layer_network = Perceptron()
multi_layer_network = MLPClassifier(max_iter=1000)

# your code here

### Exercise: k-Nearest Neighbors (kNN) classifier

This is the same kNN classifier as we saw above. Try out different values for k and the number of data points, n_samples.
Can you find a good value for k for 300 samples?

The accuracy score is in the lower right corner of the plot.
Which accuracy do you get?

In [ ]:
data3, labels3 = make_circles(n_samples = 100, noise=0.2, factor=0.5, random_state=5)

kNN_classifier = KNeighborsClassifier(10)
run_classifier(kNN_classifier, data3, labels3)